In [34]:
import time
import re
import datetime as dt
import pandas as pd
import openpyxl as xl
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm

def get_youtube_data(ent, idol):
    # 드라이버 주소 설정
    browser = webdriver.Chrome(".\chromedriver.exe") 
    browser.implicitly_wait(1)   
    
    # 크롤링 데이터를 저장할 경로 및 엑셀 파일 이름 지정
    try:      # 데이터 100개를 저장할 폴더 생성('YT_data') 및 이미 존재하는 폴더일 시 예외처리
        os.mkdir("new_YT_data") 
    except FileExistsError:
        print('YT_data 폴더가 이미 존재합니다. 해당 폴더에 데이터를 저장합니다.')
    
    result = pd.ExcelWriter(f"./new_YT_data/{ent}_{idol}.xlsx", engine='openpyxl')
    
    # 유튜브 검색을 위한 url을 만들어 열고 body 추출
    yt_url = "https://www.youtube.com"
    target_url  = yt_url + "/results?search_query="+ idol + "&sp=CAI%253D"      # 업로드날짜 순으로 필터링
    browser.get(target_url)
    body = browser.find_element(By.TAG_NAME, 'body')
    
    # 전체 데이터를 추출하기 위하여 페이지를 쭉 내림. 필터링이 되기 때문에 단순히 내리기만 하면 됨
    while True:  # 페이지 다운 수
        for pg_down in range(100):
            body.send_keys(Keys.PAGE_DOWN)
            browser.implicitly_wait(10)   
        # 해당 페이지 html 소스를 beautifulsoup을 이용하여 html 에 저장
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')
        last_date = html.find_all('span',{"class" : "inline-metadata-item style-scope ytd-video-meta-block"})[-1].get_text()
        print(last_date)
        
        # 돌리던 중 영상이 끝난 경우(데뷔가 최근인 경우)
        try:
            msg = html.find_all('yt-formatted-string',{"class" : "style-scope ytd-message-renderer"})[0].get_text()
        except:
            msg = 1
            pass
        
        if msg == '결과가 더 이상 없습니다.':
            break
        if last_date[-3:] == "년 전": 
            break
    
    # 검색 결과창에서 각 비디오 하나씩을 나누어줌
    video_datas = html.find_all('ytd-video-renderer',{'class':'style-scope ytd-item-section-renderer'})

    
    #데이터 저장을 위한 기본 데이터프레임 생성
    dataframe = pd.DataFrame({'title':[], 'youtube_url':[], 'views':[], "nice":[], "reply":[], "upload":[]})
    
    # 비디오 url을 저장할 리스트 생성(해당 url은 youtube.com/ 이후에 들어가는 부분)
    video_url_list = []
    for i in range(len(video_datas)):
        # 검색 결과 창에서 얻을 수 있는 데이터인 동영상 제목(title)과 동영상 링크 추출
        title = video_datas[i].find('a',{'id':'video-title'}).get_text() 
        url = yt_url + video_datas[i].find('a',{'id':'thumbnail'})['href']
        video_url_list.append(url) # 이후 상세 페이지 접속에 사용
        
        if url.split('/')[-2] == 'shorts': # 쇼츠의 경우 상세 페이지 속 태그 형태가 다르므로 분류
            shorts = True
        else:
            shorts = False
        
        # 빠른 크롤링을 위해 조회수가 1만(임의) 이상인 동영상만 접속하도록 함
        try: # 조회수를 감춰두어 에러가 나는 경우가 있으므로 예외처리
            view = video_datas[i].find_all("span", {"class" : "inline-metadata-item style-scope ytd-video-meta-block"})[0].get_text()
            if view[-2:] != '만회' and view[-2:] != '억회': # 조회수가 만, 억 단위가 아닌 경우
                print(view, '조회수가 적어 패스합니다.')
                continue
        except:
            pass  # continue
            
        # 상세 동영상 페이지 접속 및 바디 추출
        cur_url = video_url_list[i]
        browser.get(cur_url); browser.implicitly_wait(20)   
        body = browser.find_element(By.TAG_NAME, 'body')
        
        # 댓글 수를 보려면 페이지를 동영상 목록까지 내린 후 기다려야 나오므로 페이지를 내리고 기다리는 과정을 반복
        for pg_down in range(10):
            body.send_keys(Keys.PAGE_DOWN)
            browser.implicitly_wait(10) 
        
        # 현재 동영상 페이지 소스코드를 beautifulsoup 이용하여 html에 저장
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')
        
        # 조회수, 좋아요수, 댓글수는 영상마다 감춰두거나 막아놓는 경우가 있어 에러가 발생하여 예외처리
        try:
            if shorts == False: # 쇼츠가 아닌 일반 동영상의 경우
                print(title, shorts, end = ' ')
                
                # 상세 조회수를 크롤링하고, nnn,nnn회 라는 결과를 얻게 되어 replace로 정리하여 정수로 저장
                view_count = html.find('span',{'class':'view-count style-scope ytd-video-view-count-renderer'}).get_text().split()[1]
                view_count = view_count.replace(",", ""); view_count = view_count.replace("회", ""); view_count = int(view_count)
                print(235)
                
                # 업로드 일자를 크롤링하여 만약 현재(코드를 돌리는 시점)부터 1년 이전에 만들어진 영상이라면 break하고 다음 '아이돌'로 넘어감(업로드 일자를 기준으로 정렬된 동영상들이기 때문에 이 이후는 볼 필요가 없으므로 break)
                upload_date = html.find_all("yt-formatted-string", {"class" : "style-scope ytd-video-primary-info-renderer"})[1].get_text()
                print(upload_date, end = ' ')
                daybefore = dt.datetime.today() - dt.timedelta(days = 365) 
                   
                if upload_date.find('스트리밍') != -1 or upload_date.find('최초') != -1: # 스트리밍, 최초 공개 영상일 시 글자가 포함되어있으므로 제거
                    stream = True
                    upload_date = upload_date.split(':')[-1]
                
                if upload_date.find('.') == -1:   # 만약 업로드 날짜가 yyyy.mm.dd 형태가 아닌 경우는 24시간 이내 올라온 영상일 때 이므로 pass
                    upload_date = dt.datetime.today()
                else:
                    if list(upload_date)[-1] == ".":
                        upload_date = upload_date[:-1]
                        
                    ul_year, ul_month, ul_day = map(int,upload_date.split("."))
                    upload_date = dt.datetime(ul_year, ul_month, ul_day)
                
                d = upload_date - daybefore
                if d.days < 0:   # 업로드 1년 이전이라면 break
                    break
                
                # 좋아요 수 크롤링. 여러 태그가 있어 while
                tp = 1
                while True:
                    try:
                        print(f'type {tp}', end = ' ')
                        if tp == 1:  
                            nice = html.find_all("div", {"class" : "factoid style-scope ytd-factoid-renderer"})[0].get_text()
                            
                            break
                        elif tp == 2:
                            nices = html.find_all("div", {"id" : "segmented-like-button"})[0]
                            nice = nices.find("span", {"class" : "yt-core-attributed-string yt-core-attributed-string--white-space-no-wrap"}).get_text()

                            break
                        elif tp == 3:
                            nice = html.find_all("div", {"class" : "factoid style-scope ytd-factoid-renderer"})[0].get_text()

                            break
                        
                    except:
                        tp += 1
                        
                nice_count = nice.split()[0]
                print(tp, nice_count, end = ' ')
                
                nice_count = str(nice_count)
                
                mult = 1 # 좋아요 수가 1000개 미만인 경우는 nnn의 숫자만 반환되므로 마지막에 곱해주는 mult값을 초기화 시켜줌
                if nice_count[-1] == '천':
                    mult = 1000
                    nice_count = nice_count[:-1]
                elif nice_count[-1] == '만':
                    mult = 10000
                    nice_count = nice_count[:-1]
                elif nice_count[-1] == '억':
                    mult = 100000000
                    nice_count = nice_count[:-1]

                if nice_count.find(',') != -1:  # 수에 ,가 있으면 제거
                    nice_count = nice_count.replace(",", "")

                nice_count = float(nice_count) * mult
                
                # 댓글 수 크롤링. 댓글 수를 나타내는 태그와 class는 아래의 답글 수 등에서도 계속 사용되기 때문에 info에 댓글 수가 있는 comments-header를 추출하여 저장 후 사용
                info = html.find('h2', {'class' : 'style-scope ytd-comments-header-renderer'})
                reply_count = info.find_all('span', {'class' : 'style-scope yt-formatted-string'})[1].get_text()    #info에는 댓글/nnn/개 의 소스가 저장되어있어 2번째 인자 추출하여 get_text
                print(reply_count, end = ' ')
                
                if reply_count.find(',') != -1:    # 수에 ,가 있으면 제거
                    reply_count = reply_count.replace(",", "")
                reply_count = int(reply_count)    # 정수로 변환하여 저장
            
                # 각 데이터를 합쳐 데이터프레임을 만들고 insert_data에 저장
                insert_data = pd.DataFrame({'title':[title], 'youtube_url':[url], 'views':[view_count], "nice":[nice_count], "reply":[reply_count], "upload":[upload_date]})
                print("추출 Yes\n")  # 코드 실행 중 가시화를 위한 코드
                
            if shorts == True: # 쇼츠의 경우
                print(title, shorts, end = ' ')
                
                #쇼츠의 경우 각 쇼츠의 더보기 버튼 -> 설명 버튼을 눌러서 값을 확인해야 함
                more_btn = browser.find_elements(By.XPATH, '//*[@id="button-shape"]/button/yt-touch-feedback-shape/div/div[2]')[0]
                browser.execute_script("arguments[0].click();", more_btn)
                
                info_btn = browser.find_element(By.XPATH, '//*[@id="items"]/ytd-menu-service-item-renderer') # 설명 클릭
                browser.execute_script("arguments[0].click();", info_btn) # .click()보다 에러 적음. Element is not clickable at point 에러 방지
                
                # 현재 쇼츠 설명 페이지 소스코드를 beautifulsoup 이용하여 html에 저장
                html0 = browser.page_source
                html_info = BeautifulSoup(html0,'html.parser')
    
                # 상세 조회수를 크롤링하고, nnn,nnn 라는 결과를 얻게 되어 replace로 정리하여 정수로 저장
                view_count = html_info.find_all('yt-formatted-string',{'class':'factoid-value style-scope ytd-factoid-renderer'})[1].get_text()
                view_count = view_count.replace(",", ""); view_count = int(view_count)
                
                # 업로드 일자를 크롤링하여 만약 현재(코드를 돌리는 시점)부터 년 이전에 만들어진 영상이라면 break하고 다음 '아이돌'로 넘어감(업로드 일자를 기준으로 정렬된 동영상들이기 때문에 이 이후는 볼 필요가 없으므로 break)
                upload = html_info.find_all("div", {"class" : "factoid style-scope ytd-factoid-renderer"})[2]
                upload_date = upload["aria-label"]
                
                print(upload_date, end = ' ')
                daybefore = dt.datetime.today() - dt.timedelta(days = 365) 
                
                if upload_date.find('스트리밍') != -1 or upload_date.find('최초') != -1: # 스트리밍, 최초 공개 영상일 시 글자가 포함되어있으므로 제거
                    upload_date = upload_date.split(':')[-1]
                
                if upload_date.find('.') == -1:   # 만약 업로드 날짜가 yyyy.mm.dd 형태가 아닌 경우는 24시간 이내 올라온 영상일 때 이므로 pass
                    upload_date = dt.datetime.today()
                else:
                    if list(upload_date)[-1] == ".":
                        upload_date = upload_date[:-1]
                        
                    ul_year, ul_month, ul_day = map(int,upload_date.split("."))
                    upload_date = dt.datetime(ul_year, ul_month, ul_day)
                
                d = upload_date - daybefore
                if d.days < 0:   # 업로드 1년 이전이라면 break
                    break
                    
                # 좋아요 수 크롤링
                nice_count = html_info.find_all('yt-formatted-string',{'class':'factoid-value style-scope ytd-factoid-renderer'})[0].get_text()
                print(nice_count, end = ' ')
    
                # 좋아요 수가 100개 이하인 경우 콤마가 포함되지 않는 숫자가 반환되어 replace가 에러날 수 있으므로 문자로 변환하였다가 정제 후 정수로 다시 변환하여 저장
                nice_count = str(nice_count)
                
                mult = 1 # 좋아요 수가 1000개 미만인 경우는 nnn의 숫자만 반환되므로 마지막에 곱해주는 mult값을 초기화 시켜줌
                if nice_count[-1] == '천':
                    mult = 1000
                    nice_count = nice_count[:-1]
                elif nice_count[-1] == '만':
                    mult = 10000
                    nice_count = nice_count[:-1]
                elif nice_count[-1] == '억':
                    mult = 100000000
                    nice_count = nice_count[:-1]

                if nice_count.find(',') != -1:  # 수에 ,가 있으면 제거
                    nice_count = nice_count.replace(",", "")

                nice_count = float(nice_count) * mult
                
                # 댓글 수 크롤링. 댓글 수를 나타내는 태그와 class는 아래의 답글 수 등에서도 계속 사용되기 때문에 info에 댓글 수가 있는 comments-header를 추출하여 저장 후 사용
                reply_count = html.find_all('span', {'class' : 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap yt-core-attributed-string--text-alignment-center yt-core-attributed-string--word-wrapping'})[2].get_text()    #info에는 댓글/nnn/개 의 소스가 저장되어있어 2번째 인자 추출하여 get_text
                print(reply_count, end = ' ')
                
                if reply_count.find(',') != -1:    # 수에 ,가 있으면 제거
                    reply_count = reply_count.replace(",", "")
                reply_count = int(reply_count)    # 정수로 변환하여 저장
            
                # 각 데이터를 합쳐 데이터프레임을 만들고 insert_data에 저장
                insert_data = pd.DataFrame({'title':[title], 'youtube_url':[url], 'views':[view_count], "nice":[nice_count], "reply":[reply_count], "upload":[upload_date]})
                print("추출 Yes\n")  # 코드 실행 중 가시화를 위한 코드

        except Exception as e:
                continue 
         
        # insert_data를 위에 만들었던 dataframe에 합쳐넣고, 이를 엑셀로 변환하여 저장
        dataframe = dataframe.append(insert_data)
        dataframe.to_excel(result, index = False)
        result.save()
    
    # dataframe에 들어가는 데이터가 없는 경우, 즉 최근 1년 이내에 올라온 조회수 10000 이상의 동영상이 없는 경우 엑셀이 제대로 저장되지 않으므로 데이터프레임을 하나 만들어서 저장
    if dataframe.shape[0] == 0:
        dataframe = pd.DataFrame({'title': [0], 'youtube_url': [0], 'views':[0], "nice":[0], "reply":[0]})   # 조회수가 0인 영상을 존재하지 않기 때문에 시각화 과정에서 이를 이용하여 실제 데이터와 분리
        dataframe.to_excel(result, index = False)
        result.save()
    

def youtube_crawling():
    # 소속사와 아이돌 그룹명이 들어있는 엑셀 파일을 불러와 활성화
    raw_data = xl.load_workbook('아이돌리스트.xlsx')
    raw_data = raw_data.active
    
    # A,B 열에서 각각 소속사와 아이돌 그룹명(이 들어있는 셀 리스트)를 slice하여 저장
    enter_name = raw_data["A"][1::]
    group_name = raw_data["B"][1::]
    
    # get_youtube_data 함수에 제품명과 브랜드를 넣어 실행
    for i in range(len(group_name)):
        group = group_name[i].value   # _name[i] 자체는 셀이기 때문에 value로 셀의 내용을 추출하여 저장
        enter = enter_name[i].value
        print(f'{group}')  # 코드 실행 중 가시화를 위한 코드
        if group == '': # 엑셀 파일 내 빈 셀이 있어서 에러 나는 경우 방지
            break
        get_youtube_data(enter, group)
    
    print('추출이 완료되었습니다.')
    
if __name__=='__main__':
    youtube_crawling()
    
    

엔믹스


C:\Users\uj200\AppData\Local\Temp/ipykernel_4448/1159592095.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(".\chromedriver.exe")


YT_data 폴더가 이미 존재합니다. 해당 폴더에 데이터를 저장합니다.
8일 전
1개월 전
3개월 전
9개월 전
1년 전
조회수 3회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 6회 조회수가 적어 패스합니다.
조회수 3천회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 134회 조회수가 적어 패스합니다.
조회수 1.8천회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 353회 조회수가 적어 패스합니다.
조회수 362회 조회수가 적어 패스합니다.


[엔믹스] 무대 위에서 날라댕기는 릴욘세
 True 2023. 6. 24. 123
2.6천 50 추출 Yes

조회수 없음 조회수가 적어 패스합니다.
조회수 4.5천회 조회수가 적어 패스합니다.
조회수 4.3천회 조회수가 적어 패스합니다.
조회수 220회 조회수가 적어 패스합니다.
조회수 8.9천회 조회수가 적어 패스합니다.
조회수 6.9천회 조회수가 적어 패스합니다.
조회수 62회 조회수가 적어 패스합니다.
조회수 255회 조회수가 적어 패스합니다.


NMIXX 2차 월드투어 이슈, 그런데 이제 하루만에 끝내는... Marble O'Clock EP.1 | PICK NMIXX
 False 235
2023. 6. 24. type 1 1 5.1천 318 추출 Yes

조회수 822회 조회수가 적어 패스합니다.


[엔믹스] 선배님의 자기 PR 멘트에 웃다가 걸린 설윤 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
 False 235
2023. 6. 24. type 1 1 해당 조회수 312회 조회수가 적어 패스합니다.
조회수 356회 조회수가 적어 패스합니다.
조회수 149회 조회수가 적어 패스합니다.
조회수 173회 조회수가 적어 패스합니다.
조회수 2천회 조회수

 True 2023. 6. 2. 123
1.2천 18 추출 Yes



[있지,엔믹스] 플러팅은 얼굴로 하는 거구나
 True 2023. 6. 2. 123
1.8천 40 추출 Yes



[엔믹스] 그저 설만보 설윤
 True 2023. 6. 1. 123
8.2천 106 추출 Yes



[엔믹스] 똑같은 댓글 읽는 오해원 설윤아 ㅋㅋㅋㅋ
 True 2023. 6. 1. 123
11만 479 추출 Yes



[엔믹스] 공부 자극 영상
 True 2023. 6. 1. 123
4.9천 63 추출 Yes



[엔믹스] 아샷추는 못 참는 규진이
 True 2023. 5. 31. 123
1.4만 115 추출 Yes



[엔믹스] 지우를 엄청나게 귀여워하는 설윤이 ㅋㅋㅋㅋ
 True 2023. 5. 31. 123
2.4만 151 추출 Yes



[엔믹스] 아재개그에 진심인 오해원 ㅋㅋㅋㅋ
 True 2023. 5. 30. 123
2.2만 164 추출 Yes



[엔믹스] 창문 닦기 장인
 True 2023. 5. 29. 123
6.1천 65 추출 Yes



[엔믹스] 설레는 기분을 몸으로 표현하는 엔믹스 ㅋㅋㅋㅋ
 True 2023. 5. 29. 123
4.8천 91 추출 Yes



엔믹스 지우 살 빠졌다가 또 심각하게 쪘다 망했다 (이슈) [장원영보다 두배]
 False 235
2023. 5. 29. type 1 1 해당 

[엔믹스] 오해원 레전드 장면 ㅋㅋㅋㅋㅋ
 True 2023. 5. 29. 123
3.7천 29 추출 Yes



해는 원 모양이다#엔믹스 #해원 #유화
 True 2023. 5. 28. 123
895 141 추출 Yes



[4K60P]230527 엔믹스(NMIXX) - Love Me Like This 설윤 직캠 fancam @2023 드림콘서트
 False 235
2023. 5. 28. type 1 1 581 10 추출 Yes



[엔믹스] 릴리x해원 데뷔 전 레전드 커버(Destiny's Child-Survivor)
 True

조회수 3.2천회 조회수가 적어 패스합니다.


엔믹스 무대 연습 댓글모음 "Love Me Like This"
 False 235
2023. 4. 5. type 1 1 6.7천 248 추출 Yes



단체로 비주얼폼 미친 엔믹스 비주얼캠 #비주얼캠 #visualcam #NMIXX #Love_Me_Like_This
 True 2023. 4. 5. 123
9.3만 441 추출 Yes



[페이스캠4K] 엔믹스 설윤 'Love Me Like This' (NMIXX SULLYOON FaceCam) @SBS Inkigayo 230402
 False 235
2023. 4. 4. type 1 1 7천 123 추출 Yes



[엔믹스] 실력파 아이돌이 라이브 연습하는 방법
 True 2023. 4. 4. 123
5.1만 360 추출 Yes



[NMIXX] “Young, Dumb, Stupid” Stage Practice
 False 235
2023. 4. 4. type 1 1 5.5만 1,557 추출 Yes



JYP막내의 필수조건 [트와이스,있지,엔믹스]
 True 2023. 4. 3. 123
2.2천 34 추출 Yes



[엔믹스] 무도 밈 하나 더 추가하는 오해원 ㅋㅋㅋㅋ
 True 2023. 4. 3. 123
3.6만 253 추출 Yes



[덕질캡처용♥CLEAN] 엔믹스 - 러브 미 라이크 디스 (NMIXX - Love Me Like This) l #쇼챔피언┃EP.469
 False 235
2023. 4. 3. type 1 1 3.8천 58 추출 Yes



[안방1열 직캠4K] 엔믹스 설윤 'Love Me Like This' (NMIXX SULLYOON FanCam) @SBS Inkigayo 230402
 False 235
2023. 4. 3. type 1 1 1.8만 628 추출 Yes



NMIXX.zip 📂 O.O부터 Love Me Like This까지 | Show! MusicCore
 False 235
2023. 4. 3. type 1 1 

...



230101 SUPER JUNIOR (슈퍼주니어) - Black Suit + Sorry Sorry + Mango
 False 235
2023. 1. 1. type 1 1 294 7 추출 Yes



Super Junior Stage at SMCU 2023 - Danger, Black Suit, Sorry Sorry & Mango
 False 235
2023. 1. 1. type 1 1 5.8천 187 추출 Yes



SUPER JUNIOR 슈퍼주니어 'Celebrate' (SMTOWN LIVE 2023 : SMCU PALACE @ KWANGYA)
 False 235
2023. 1. 1. type 1 1 1.6천 41 추출 Yes



[2022 MBC 가요대제전] 조수미 x 슈퍼주니어 려욱 - 아이 러브 서울, MBC 221231 방송
 False 235
2023. 1. 1. type 1 1 1.4천 119 추출 Yes



[ENG][아형✪하이라이트] MZ와 가까워지기.. 아니 멀어지기 💙슈주💙와 함께 우당탕탕 2022 기말고사🤣 | 아는 형님 | JTBC 221231 방송
 False 235
2022. 12. 31. type 1 1 1.1만 478 추출 Yes



웅장 그 자체💪 슈퍼주니어 콘서트 오프닝 곡 〈Superman〉♬ | 아는 형님 364회 | JTBC 221231 방송
 False 235
2022. 12. 31. type 1 1 978 18 추출 Yes



[골라봐야지][슈퍼주니어 레전드 스페셜♨] 별별★ 얘기 다 하는 슈주 토크 무한 제공｜아는형님｜JTBC 221224 방송 외
 False 235
2022. 12. 28. type 1 1 1.4만 617 추출 Yes



탈퇴가 걸린 슈퍼주니어의 매운맛 지목 게임😱🔥
 False 235
2022. 12. 28. type 1 1 5.2천 221 추출 Yes

조회수 7.3천회 조회수가 적어 패스합니다.


Come Back #슈퍼주니어 17년 우정이 Very Hot!!🔥🔥 [대케가수] 

조회수 9.8천회 조회수가 적어 패스합니다.
조회수 3.7천회 조회수가 적어 패스합니다.
조회수 6.6천회 조회수가 적어 패스합니다.




SUPER JUNIOR 슈퍼주니어 'Mango' 안무 연습 비하인드 (With Aiki)
 False 235
2022. 7. 14. type 1 1 3.2만 991 추출 Yes



SUPER JUNIOR Always Lyrics (슈퍼주니어 Always 가사) [Color Coded Lyrics Han/Rom/Eng]
 False 235
2022. 7. 13. type 1 1 해당 

SUPER JUNIOR My Wish Lyrics (슈퍼주니어 My Wish 가사) [Color Coded Lyrics Han/Rom/Eng]
 False 235
2022. 7. 13. type 1 1 해당 

SUPER JUNIOR Don't Wait Lyrics (슈퍼주니어 Don't Wait 가사) [Color Coded Lyrics Han/Rom/Eng]
 False 235
2022. 7. 13. type 1 1 해당 

[LIVE] SUPER JUNIOR (슈퍼주니어) - Mangoㅣstage&FLO:취향의 발견ㅣStudio FLO
 False 235
2022. 7. 13. type 1 1 3.1만 2,399 추출 Yes



[Full Album] SUPER JUNIOR (슈퍼주니어) - The Road: Keep on Going
 False 235
2022. 7. 12. type 1 1 2.5천 75 추출 Yes



SUPER JUNIOR 슈퍼주니어 'Mango' MV
 False 235
최초 공개: 2022. 7. 12. type 1 type 2 2 37만 66,128 추출 Yes

조회수 817회 조회수가 적어 패스합니다.
조회수 2.1천회 조회수가 적어 패스합니다.
조회수 2.6천회 조회수가 적어 패스합니다.
조회수 4.6천회 조회수가 적어 패스합니다.
조회수 3.3천회 조회수가 적어 패스합니다.
조회수 1.7천회 조회

C:\Users\uj200\AppData\Local\Temp/ipykernel_4448/1159592095.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(".\chromedriver.exe")


YT_data 폴더가 이미 존재합니다. 해당 폴더에 데이터를 저장합니다.
1개월 전
10개월 전
1년 전
조회수 512회 조회수가 적어 패스합니다.
조회수 70회 조회수가 적어 패스합니다.
조회수 51회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 20회 조회수가 적어 패스합니다.
조회수 14회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 553회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 120회 조회수가 적어 패스합니다.
조회수 171회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 5.5천회 조회수가 적어 패스합니다.
조회수 6.2천회 조회수가 적어 패스합니다.
조회수 277회 조회수가 적어 패스합니다.
조회수 333회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 187회 조회수가 적어 패스합니다.
조회수 1.5천회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 49회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 881회 조회수가 적어 패스합니다.
조회수 57회 조회수가 적어 패스합니다.
조회수 8회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 605회 조회수가 적어 패스합니다.
조회수 7회 조회수가 적어 패스합니다.
조회수 378회 조회수가 적어 패스합니다.
조회수 없음 조회수가 적어 패스합니다.
조회수 4천회 조회수가 적어 패스합니다.
조회수 21회 조회수가 적어 패스합니다.
조회수 1.2천회 조회수가 적어 패스합니다.


10년이 지나도 소녀시대 서현이 태연에게 미안한 이유 ( ft. 칼꿈치 피해자 ) #shorts
 True 2023. 6. 23. 123
845 14 추출 Yes

조회수 176회 조회수가 적

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00D08893+48451]
	(No symbol) [0x00C9B8A1]
	(No symbol) [0x00BA5058]
	(No symbol) [0x00B8D073]
	(No symbol) [0x00BEDEBB]
	(No symbol) [0x00BFBFD3]
	(No symbol) [0x00BEA0B6]
	(No symbol) [0x00BC7E08]
	(No symbol) [0x00BC8F2D]
	GetHandleVerifier [0x00F68E3A+2540266]
	GetHandleVerifier [0x00FA8959+2801161]
	GetHandleVerifier [0x00FA295C+2776588]
	GetHandleVerifier [0x00D92280+612144]
	(No symbol) [0x00CA4F6C]
	(No symbol) [0x00CA11D8]
	(No symbol) [0x00CA12BB]
	(No symbol) [0x00C94857]
	BaseThreadInitThunk [0x76EB00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77217B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77217B1E+238]
